In [1]:
import pandas as pd
import torch
from torch.utils import data
from d2l import torch as d2l

In [2]:
true_w = torch.tensor([2,-3.4])
true_b = 4.2
features,labels = d2l.synthetic_data(true_w,true_b,1000)   
# 之前的生成数据的方式一个封装，相当于围绕真实的w，b生成一组扰动的数据(一千个)，平时也可以用到

In [3]:
# 生成线性回归的扰动数据的情况
# fea = features.numpy()
# Data = pd.DataFrame(fea.tolist())
# Data.to_excel("从tensor转化为excel.xlsx",index=False)
# 思考一下怎么去生成更多的变量的扰动数据

在深度学习中怎么去读取数据集

In [4]:
def load_array(data_arrays,batch_size,is_train = True):
    """Pytorch中数据迭代器的构造"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset,batch_size,shuffle=is_train)   # 每个epoch要不要重新洗牌
# 这里is_train的意义是什么？

In [5]:
batch_size = 10      # 每次操作中一次的数量
data_iter = load_array((features,labels),batch_size)

In [6]:
# 使用iter构造python迭代器，并且使用next从迭代器中获取第一项
next(iter(data_iter))

[tensor([[ 0.9673, -1.8430],
         [-0.2685,  1.0562],
         [ 1.4011, -0.0946],
         [ 1.2214, -1.9569],
         [-0.4392, -0.1399],
         [-0.4990,  0.9791],
         [-1.0206,  1.0568],
         [ 0.3724,  0.6013],
         [-0.1423,  0.4120],
         [-0.0350,  0.6601]]),
 tensor([[12.3855],
         [ 0.0814],
         [ 7.3046],
         [13.3014],
         [ 3.7905],
         [-0.1293],
         [-1.4434],
         [ 2.9059],
         [ 2.4975],
         [ 1.8929]])]

在$Pytorch$中，全连接层在Linear类中定义

要向`nn.Linear`中传递两个参数
一个是$输入特征的形状$
另一个是$输出特征的形状$

In [7]:
from torch import nn
net = nn.Sequential(nn.Linear(2,1))   # 这里的(2,1)即是输入特征的形状和输出特征的形状

在使用net之前，我们需要对模型的参数进行一个初始化

In [8]:
# 通过net[0]选择网络中的第一个图层
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [9]:
# 定义损失函数，平方2阶范数，默认情况下将返回所有样本损失的平均值
loss = nn.MSELoss()

小批量随机梯度下降**MBGD**算法是一种优化神经网络的工具

$PyTorch$在`optim`模块中实现了该算法的很多变种

对于MBGD算法只要指定学习率lr值

In [10]:
trainer = torch.optim.SGD(net.parameters(),lr=0.03)

在每个迭代周期里，我们将完整遍历一次数据集(train_data)，不停地从中获取一个小批量的输入和响应的标签

对于每一个**小批量**

- 通过调用`net(X)`生成预测并计算损失l(前向传播$forward$)
- 通过进行反向传播($backward$)来计算**梯度**
- 通过调用优化器来更新模型的参数

In [11]:
num_epochs = 3
for epoch in range(num_epochs):
    for X,y in data_iter:
        l = loss(net(X),y)
        trainer.zero_grad()   # 梯度清零
        l.backward()          # 反向传播
        trainer.step()        # 优化器对x(X)的值进行更新，x=x-lr*x.grad 沿着梯度的反方向调整变量值
    l = loss(net(features),labels)
    print(f'epoch{epoch+1},loss{l:f}')

epoch1,loss0.000213
epoch2,loss0.000104
epoch3,loss0.000104


In [13]:
w = net[0].weight.data
print('w的估计误差:',true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差:',true_b - b)

w的估计误差: tensor([-1.6689e-05,  7.0095e-04])
b的估计误差: tensor([0.0003])
